In [ ]:
!pip install cx_Oracle

In [1]:
import os, argparse, sys
import cx_Oracle                                                                         
import time
import sys
import sqlite3
import os
import numpy as np

## Fill in your username and password
clarity_user = ''
clarity_psw = ''

## if 'exaorat2' does not work, try 'exaorav2'
dsnStr = cx_Oracle.makedsn("exaorat2-scan.chop.edu", "1521", service_name="CLTprd_pri.chop.edu")
remote_con = cx_Oracle.connect(user = clarity_user, password = clarity_psw, dsn = dsnStr)
remote_cur = remote_con.cursor()

### Collect patient IDs by icd codes

sql command to query Down Syndrome patients (ICD 10: Q90.9; ICD 9: 758.0) and their gender and birthday

In [6]:

remote_cmd = """

with ds as   (select dx.pat_id,dx.dx_id
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd9 icd on icd.dx_Id = dx.dx_id
                where icd.code like '758.0%'

                union
                select dx.pat_id, dx.dx_id
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd10 icd on icd.dx_Id = dx.dx_id
                where icd.code like 'Q90.9%'

                union
                select h.pat_id, hdx.dx_id
                from hsp_acct_dx_List hdx
                inner join pat_enc_hsp h on h.hsp_account_id = hdx.hsp_account_id
                inner join clarity_edg edg on edg.dx_Id = hdx.dx_Id
                where (edg.ref_bill_code like '758.0%' or edg.ref_bill_code like 'Q90.9%')
                and edg.ref_bill_code_set_c in (1,2)
                -- ref_bill_code_set_c: 1 is ICD-9-CM and 2 is ICD-10-CM

                union
                select pl.pat_id,pl.dx_id
                from problem_list pl
                inner join edg_current_icd9 icd on icd.dx_Id = pl.dx_id
                where icd.code like '758.0%'
                and  (pl.problem_status_c is null or pl.problem_status_c <> 3)
                -- problem_status_c: 3 is deleted

                union
                select pl.pat_id,pl.dx_id
                from problem_List pl
                inner join edg_current_icd10 icd on icd.dx_Id = pl.dx_id
                where icd.code like 'Q90.9%'
                 and  (pl.problem_status_c is null or pl.problem_status_c <> 3)

                 ) -- inclusion diagnoses
                 
select distinct(p.pat_id), p.sex_c, p.birth_date
        from patient p
        inner join ds on ds.pat_id = p.pat_id
        inner join zc_sex  sex on sex.rcpt_mem_sex_c = p.sex_c
"""
### execute the sql command
remote_cur.execute(remote_cmd)

### fetch all the data 
data = remote_cur.fetchall()

### num of queried patients (as on 12/14/2020)
len(data)

DatabaseError: ORA-00904: "ICD"."CODE": invalid identifier

## store the data in the asd_dict ('dict' is 'hashkey' in other languages)

In [4]:


asd_dict = dict()

for d in data:
    pat_id = d[0]
    sex = d[1]
    birth = d[2]
    asd_dict[pat_id] = [sex,birth]
    
### num of queried patients (as on 1/25/2021)    
len(asd_dict)

4417

In [2]:
nonds_nonpsy_pat = os.listdir('/mnt/isilon/wang_lab/shared/datasets/nonds_nonpsy_hpo_data')

### Query the ICD

In [3]:
pat_icd_dict = dict()

In [4]:

remote_cmd = """

select dx.pat_id,dx.dx_id, icd.code 
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd9 icd on icd.dx_Id = dx.dx_id
                where dx.pat_id= '{}'

                union
                select dx.pat_id, dx.dx_id, icd.code 
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd10 icd on icd.dx_Id = dx.dx_id
                where dx.pat_id= '{}'

                union
                select h.pat_id, hdx.dx_id, edg.ref_bill_code 
                from hsp_acct_dx_List hdx
                inner join pat_enc_hsp h on h.hsp_account_id = hdx.hsp_account_id
                inner join clarity_edg edg on edg.dx_Id = hdx.dx_Id
                where h.pat_id= '{}'
                and edg.ref_bill_code_set_c in (1,2)
                -- ref_bill_code_set_c: 1 is ICD-9-CM and 2 is ICD-10-CM

                union
                select pl.pat_id,pl.dx_id, icd.code 
                from problem_list pl
                inner join edg_current_icd9 icd on icd.dx_Id = pl.dx_id
                where pl.pat_id= '{}'
                and  (pl.problem_status_c is null or pl.problem_status_c <> 3)
                -- problem_status_c: 3 is deleted

                union
                select pl.pat_id,pl.dx_id, icd.code 
                from problem_List pl
                inner join edg_current_icd10 icd on icd.dx_Id = pl.dx_id
                where pl.pat_id= '{}'
                 and  (pl.problem_status_c is null or pl.problem_status_c <> 3)


"""

In [6]:
pcnt = 0

for p in nonds_nonpsy_pat:
    pcnt += 1
    ### execute the sql command
    remote_cur.execute(remote_cmd.format(p,p,p,p,p))

    ### fetch all the data 
    data = remote_cur.fetchall()

    ### num of queried patients (as on 1/25/2021)
    len(data)
    
    icd_set = set()
    
    for d in data:
        icd_set.add(d[2])
        
    pat_icd_dict[p] = icd_set
    
    
    if(pcnt % 500 ==0): print('queried {} patients ICD code.'.format(str(pcnt)))

queried 500 patients ICD code.
queried 1000 patients ICD code.
queried 1500 patients ICD code.
queried 2000 patients ICD code.
queried 2500 patients ICD code.
queried 3000 patients ICD code.
queried 3500 patients ICD code.
queried 4000 patients ICD code.


In [8]:
file_name = '/mnt/isilon/wang_lab/shared/datasets/DownSyndromeData/nonDS_nonPsy_patients_icds.tsv'

with open(file_name, 'w+') as fw:
    for p in pat_icd_dict.keys():
        fw.write('{}\t{}\n'.format(p, '|'.join(pat_icd_dict[p])))

### Query ICD of all patients

In [2]:

remote_cmd = """

select dx.pat_id,dx.dx_id, icd.code 
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd9 icd on icd.dx_Id = dx.dx_id


                union
                select dx.pat_id, dx.dx_id, icd.code 
                from pat_enc_dx dx
                inner join pat_enc e on e.pat_enc_Csn_id = dx.pat_enc_Csn_id
                inner join edg_current_icd10 icd on icd.dx_Id = dx.dx_id


                union
                select h.pat_id, hdx.dx_id, edg.ref_bill_code 
                from hsp_acct_dx_List hdx
                inner join pat_enc_hsp h on h.hsp_account_id = hdx.hsp_account_id
                inner join clarity_edg edg on edg.dx_Id = hdx.dx_Id

                and edg.ref_bill_code_set_c in (1,2)
                -- ref_bill_code_set_c: 1 is ICD-9-CM and 2 is ICD-10-CM

                union
                select pl.pat_id,pl.dx_id, icd.code 
                from problem_list pl
                inner join edg_current_icd9 icd on icd.dx_Id = pl.dx_id

                and  (pl.problem_status_c is null or pl.problem_status_c <> 3)
                -- problem_status_c: 3 is deleted

                union
                select pl.pat_id,pl.dx_id, icd.code 
                from problem_List pl
                inner join edg_current_icd10 icd on icd.dx_Id = pl.dx_id

                 and  (pl.problem_status_c is null or pl.problem_status_c <> 3)


"""

In [10]:
local_db = '../pat_icd.db'

In [19]:

local_con = sqlite3.connect(local_db)
local_cur = local_con.cursor()

In [21]:
### Create tables
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
    finally:
        if c:
            c.close()

## sql command to create the table to store patients' data (pat_id, gender, birthdays)
pat_icd_table = """CREATE TABLE IF NOT EXISTS ICD (
                                        pat_id text,
                                        icd text
                                        
                                    ); """


create_table(local_con, pat_icd_table)
local_con.commit()

In [13]:
def local_sql_insert(conn, insert_cmd, insert_data):
    """
    Create a new task
    :param conn:
    :return:
    """
    cur = conn.cursor()
    cur.execute(insert_cmd, insert_data)
    #conn.commit()
    cur.close()
    return cur.lastrowid


In [14]:
### execute the sql command
remote_cur.execute(remote_cmd)

### fetch all the data 
data = remote_cur.fetchall()

In [15]:
err_log = 'icd_query.err.0211'

In [22]:
for d in data:
    pat_id = d[0]
    icd = d[2]
    try:    
        local_insert_cmd_patients = ''' INSERT INTO ICD (pat_id, icd) VALUES(?, ?) '''
        local_insert_data = (pat_id,icd)
        local_sql_insert(local_con, local_insert_cmd_patients,local_insert_data )
    ## if it meets an error, the error message is written down in a log file
    except Exception as e:
        print(e)
        fa = open("err_log", 'a+')
        fa.write("Error in {}'\n".format(str(d) ))
        fa.write(str(e))
        fa.write('\n')
        fa.close()
        continue
    

In [23]:
local_con.commit()

In [18]:
local_con.close()